



# **IP2_Run2**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))



### **Generate Mathematical Model**

In [ ]:
problem = """Your goal is to invest in several of 10 possible investment strategies in the most optimal way. The historic returns of those strategies are stored in the file "investments_data.csv". Each column represents one strategy and the rows are the past investment outcomes. There is no index and the values are separated by a ;.

The costs for investing in a given investment is stored in a vector A, which has one value for each strategy in order.
The values are: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]

You can only invest once into an investment.

Unfortunately due to other costs and inflation, your available budget at this time is uncertain. There are four possible budget scenarios with different probabilities: scenario 1 with 1000 euros and probability of 0.55, scenario 2  with 1100 euros and probability of 0.4, scenario 3 with 900 euros and probability of 0.04, scenario 4 with 1200 euros and probability of 0.01.
The tolerable probability of exceeding the budget is 0.4.

Please formulate a mean-variance mathematical model for this optimization problem, considering the past performance of investment strategies and the uncertain budget. You can take 2 as the risk parameter r. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [ ]:
print(response1.choices[0].message.content)

Decision variables:

Let:
xi = 1 if the investment in strategy i is made, 0 otherwise, for i in {1,..,10}

Parameters:

Ci = cost of investment strategy i, for i in {1,...,10}
Ri = mean return of investment strategy i, for i in {1,...,10}
Vi = variance of return of investment strategy i, for i in {1,...,10}
Bj = budget in scenario j, for j in {1,...,4}
Pj = probability of scenario j, for j in {1,...,4}
T = tolerable probability of exceeding the budget (given as 0.4)
r = risk parameter (given as 2)

We can derive the mean return (Ri) and variance of return (Vi) for each investment strategy i, from the data file "investments_data.csv".

Objective function:

Maximize Σ Ri*xi - r*Σ Vi*xi^2
i ranges from 1 to 10

Constraints:

1. You can only invest once in each strategy:
xi ∈ {0, 1} for all i in {1,...,10}

2. The chance of exceeding the budget should not exceed the tolerable probability:
Σ Pj*(Σ Ci*xi - Bj) ≤ T 
i ranges from 1 to 10; j ranges from 1 to 4. Please note, the expression with

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

NameError: name 'response1' is not defined

In [ ]:
Markdown(response2.choices[0].message.content)

Here is a Python script utilizing the Pyomo module to represent the mathematical model you outlined. In this example, we will focus on 5 investment strategies (instead of 10) and 2 scenarios (instead of 4) for simplicity.

```python
from pyomo.environ import *

# Sample data
strategies = ['Strategy1', 'Strategy2', 'Strategy3', 'Strategy4', 'Strategy5']
scenarios = ['Scenario1', 'Scenario2']

C = {'Strategy1': 3, 'Strategy2': 4, 'Strategy3': 5, 'Strategy4': 6, 'Strategy5': 7} # cost
R = {'Strategy1': 8, 'Strategy2': 9, 'Strategy3': 10, 'Strategy4': 11, 'Strategy5': 12} # return
V = {'Strategy1': 2, 'Strategy2': 3, 'Strategy3': 4, 'Strategy4': 1, 'Strategy5': 3} # variance

B = {'Scenario1': 20, 'Scenario2': 15} # budget
P = {'Scenario1': 0.6, 'Scenario2': 0.4} # probabilities
T = 0.4 # tolerable probability
r = 2   # risk parameter

m = ConcreteModel()

# Sets
m.i = Set(initialize=strategies) 
m.j = Set(initialize=scenarios)

# Variables
m.x = Var(m.i, within=Binary)

# Objective
m.OBJ = Objective(expr = sum(R[i]*m.x[i] - r*V[i]*m.x[i] for i in m.i),
                  sense = maximize)

# Constraints
m.Const1 = Constraint(expr = sum(P[j]*(sum(C[i]*m.x[i] for i in m.i) - B[j]) for j in m.j) <= T)
m.Const2 = Constraint(m.j, rule=lambda m, j: sum(C[i]*m.x[i] for i in m.i) <= B[j])

# Solving the model
solver = SolverFactory('glpk')
solver.solve(m)

# Printing the results
for i in m.i:
    print('Investment strategy:', i, ' Investment decision:', m.x[i].value)
```        

This script first sets up sample data for the 5 investment strategies and 2 scenarios. It then creates a ConcreteModel object and adds the sets, decision variables, objective function, and constraints to the model. It tries to maximize the formulated objective function within the given constraints by using the GLPK solver. Lastly, it prints out the decision to invest or not in each investment strategy. The decision is a binary value - 1 if the investment is made and 0 otherwise.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# Sample data
strategies = ['Strategy1', 'Strategy2', 'Strategy3', 'Strategy4', 'Strategy5']
scenarios = ['Scenario1', 'Scenario2']

C = {'Strategy1': 3, 'Strategy2': 4, 'Strategy3': 5, 'Strategy4': 6, 'Strategy5': 7} # cost
R = {'Strategy1': 8, 'Strategy2': 9, 'Strategy3': 10, 'Strategy4': 11, 'Strategy5': 12} # return
V = {'Strategy1': 2, 'Strategy2': 3, 'Strategy3': 4, 'Strategy4': 1, 'Strategy5': 3} # variance

B = {'Scenario1': 20, 'Scenario2': 15} # budget
P = {'Scenario1': 0.6, 'Scenario2': 0.4} # probabilities
T = 0.4 # tolerable probability
r = 2   # risk parameter

m = ConcreteModel()

# Sets
m.i = Set(initialize=strategies)
m.j = Set(initialize=scenarios)

# Variables
m.x = Var(m.i, within=Binary)

# Objective
m.OBJ = Objective(expr = sum(R[i]*m.x[i] - r*V[i]*m.x[i] for i in m.i),
                  sense = maximize)

# Constraints
m.Const1 = Constraint(expr = sum(P[j]*(sum(C[i]*m.x[i] for i in m.i) - B[j]) for j in m.j) <= T)
m.Const2 = Constraint(m.j, rule=lambda m, j: sum(C[i]*m.x[i] for i in m.i) <= B[j])

# Solving the model
solver = SolverFactory('glpk')
solver.solve(m)

# Printing the results
for i in m.i:
    print('Investment strategy:', i, ' Investment decision:', m.x[i].value)

Investment strategy: Strategy1  Investment decision: 1.0
Investment strategy: Strategy2  Investment decision: 1.0
Investment strategy: Strategy3  Investment decision: 0.0
Investment strategy: Strategy4  Investment decision: 1.0
Investment strategy: Strategy5  Investment decision: 0.0


In [ ]:
from pyomo.environ import *
import pandas as pd
#Start - data inputted by human
strategies = ['Strategy1', 'Strategy2', 'Strategy3', 'Strategy4', 'Strategy5', 'Strategy6', 'Strategy7', 'Strategy8', 'Strategy9', 'Strategy10']
scenarios = ['Scenario1', 'Scenario2',  'Scenario3',  'Scenario4']
C = {'Strategy1': 80, 'Strategy2': 340, 'Strategy3': 410, 'Strategy4': 50, 'Strategy5': 180, 'Strategy6': 221, 'Strategy7': 15, 'Strategy8': 348, 'Strategy9': 191, 'Strategy10': 225} # cost


df = pd.read_csv("investments_data.csv",sep=";", header=None)
headers = []
for i in range(len(df.columns)):
    headers.append("Strategy"+str((i+1)))
df.columns = headers

R = df.mean(0).to_dict()
V = df.var(0).to_dict()


B = {'Scenario1': 1000, 'Scenario2': 1100, 'Scenario3': 900, 'Scenario4': 1200} #budget
P = {'Scenario1': 0.55, 'Scenario2': 0.4, 'Scenario3': 0.04, 'Scenario4': 0.01} #probabilities
#end
T = 0.4 # tolerable probability
r = 2 # risk parameter

m = ConcreteModel()

# Sets
m.i = Set(initialize=strategies)
m.j = Set(initialize=scenarios)

# Variables
m.x = Var(m.i, within=Binary)

# Objective
m.OBJ = Objective(expr = sum(R[i]*m.x[i] - r*V[i]*m.x[i] for i in m.i),
                  sense = maximize)

# Constraints
m.Const1 = Constraint(expr = sum(P[j]*(sum(C[i]*m.x[i] for i in m.i) - B[j]) for j in m.j) <= T)
m.Const2 = ConstraintList()
for j in m.j:
    m.Const2.add(expr=sum(C[i] * m.x[i] for i in m.i) <= B[j])

# Solving the model
solver = SolverFactory('glpk')
solver.solve(m)

# Printing the results
for i in m.i:
    print('Investment strategy:', i, ' Investment decision:', m.x[i].value)
print(m.OBJ())

Investment strategy: Strategy1  Investment decision: 0.0
Investment strategy: Strategy2  Investment decision: 0.0
Investment strategy: Strategy3  Investment decision: 1.0
Investment strategy: Strategy4  Investment decision: 0.0
Investment strategy: Strategy5  Investment decision: 0.0
Investment strategy: Strategy6  Investment decision: 1.0
Investment strategy: Strategy7  Investment decision: 0.0
Investment strategy: Strategy8  Investment decision: 0.0
Investment strategy: Strategy9  Investment decision: 0.0
Investment strategy: Strategy10  Investment decision: 1.0
188.47780903195854


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**